<a href="https://colab.research.google.com/github/Takumi173/2023Test/blob/main/Coding_2_CreateModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#事前設定

In [ ]:
# OK installation
!pip install -q transformers fugashi[unidic-lite]
!pip install transformers[torch]
!pip install mecab-python3
!pip install unidic-lite
!pip install janome
!pip install lime
!export MECABRC='/etc/mecabrc'
!pip install mojimoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.9/600.9 kB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 101.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 70.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lim

In [ ]:
import MeCab
import random
import mojimoji

In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers import Trainer, TrainingArguments
from torch.optim import AdamW
import numpy as np
import time
import os
import torch
import unicodedata

# Training data読み込み

xxx_wakati.tsvは前処理で品詞を絞ったデータ、xxx_wakati_all.tsvは前処理で記号のみ削除したデータ。

In [ ]:
#事前に分かち書き&正規化されたテキストを使用する
!wget https://github.com/Takumi173/2023Test/releases/download/20231104/combined_training_data_watati.tsv
!wget https://github.com/Takumi173/2023Test/releases/download/20231104/combined_training_data_watati_all.tsv
#!wget https://github.com/Takumi173/2023Test/releases/download/20231104/Test_Data_Wakati.tsv
#!wget https://github.com/Takumi173/2023Test/releases/download/20231104/Test_Data_Wakati_all.tsv

In [ ]:

#tmp = pd.read_table('combined_training_data_watati.tsv')
tmp = pd.read_table('combined_training_data_watati_all.tsv')

tmp["ICD10_Term"] = tmp["ICD10_Term_WAKATI"]

df = tmp[["ICD10_CATEGORY", "ICD10_Term", "FOR_USAGE"]]
df

,ICD10_CATEGORY,ICD10_Term,FOR_USAGE
0,A00,コレラ 菌 による コレラ,train
1,A00,エルトールコレラ 菌 による コレラ,train
2,A00,コレラ 詳細不明,train
3,A01,腸チフス,train
4,A01,パラチフス A,train
...,...,...,...
136872,X99,施設 サービス 及び 商業 死亡 及び 傷害 もとづく に 加害 よる に 物体 な 鋭利,train
136873,X99,現場 建築 及び 地域 用 工業 死亡 及び 傷害 もとづく に 加害 よる に 物体 な 鋭利,train
136874,X99,農場 死亡 及び 傷害 もとづく に 加害 よる に 物体 な 鋭利,train
136875,X99,場所 たれる さ 明示 の 他 その 死亡 及び 傷害 もとづく に 加害 よる に 物体 ...,train


# データ確認と分割

In [ ]:
!mkdir output

In [ ]:
df2 = df.drop_duplicates().reset_index(drop = True)

code_set = set(df2["ICD10_CATEGORY"].tolist())
code_list = list(code_set)
df2["id"] = df2["ICD10_CATEGORY"].apply(lambda x: code_list.index(x))

print(df2.shape)
df2.to_csv("Output_df_csv.tsv",sep = "\t", index=False)
df2

(125787, 4)


,ICD10_CATEGORY,ICD10_Term,FOR_USAGE,id
0,A00,コレラ 菌 による コレラ,train,160
1,A00,エルトールコレラ 菌 による コレラ,train,160
2,A00,コレラ 詳細不明,train,160
3,A01,腸チフス,train,241
4,A01,パラチフス A,train,241
...,...,...,...,...
125782,X99,施設 サービス 及び 商業 死亡 及び 傷害 もとづく に 加害 よる に 物体 な 鋭利,train,868
125783,X99,現場 建築 及び 地域 用 工業 死亡 及び 傷害 もとづく に 加害 よる に 物体 な 鋭利,train,868
125784,X99,農場 死亡 及び 傷害 もとづく に 加害 よる に 物体 な 鋭利,train,868
125785,X99,場所 たれる さ 明示 の 他 その 死亡 及び 傷害 もとづく に 加害 よる に 物体 ...,train,868


In [ ]:
# Make id and term list

# 3 digit
target_column="ICD10_CATEGORY"
# 4 digit
#target_column="ICD10_Code"

df_id_term = df2.copy()
df_id_term = df_id_term[["id", target_column]]
df_id_term = df_id_term.drop_duplicates().reset_index(drop = True)
df_id_term.to_csv("./output/Output_id_term.csv", index=False)
print(df_id_term.shape)
df_id_term


(1529, 2)


,id,ICD10_CATEGORY
0,160,A00
1,241,A01
2,970,A02
3,1377,A03
4,1261,A04
...,...,...
1524,549,X95
1525,520,X96
1526,186,X97
1527,263,X98


In [ ]:
#Training Test data 分割
import csv
from sklearn.model_selection import train_test_split

docs = df2["ICD10_Term"].tolist()
labels = df2["id"].tolist()

# サンプル数が2未満のクラスを除外
classes_to_exclude = [cls for cls in set(labels) if list(labels).count(cls) < 2]
docs_filtered = []
labels_filtered = []
for i in range(len(labels)):
    if labels[i] not in classes_to_exclude:
        docs_filtered.append(docs[i])
        labels_filtered.append(labels[i])

# データをトレーニングセットとテストセットに分割（stratifyを使用）
train_docs, test_docs, train_labels, test_labels =  train_test_split(docs_filtered, labels_filtered, test_size=0.2, random_state=42, stratify=labels_filtered)



In [ ]:
print(len(train_docs))
print(len(test_docs))

100629
25158


In [ ]:
from collections import Counter
import statistics

def count_labels(lst):
  sample_list = lst

  item_counts = Counter(sample_list)


  # 最大値（出現回数が最も多い項目）を取得
  max_item = max(item_counts, key=item_counts.get)
  max_count = item_counts[max_item]

  # 最小値（出現回数が最も少ない項目）を取得
  min_item = min(item_counts, key=item_counts.get)
  min_count = item_counts[min_item]

  print(f"最大値のID: {max_item} (出現回数: {max_count})")
  print(f"最小値のID: {min_item} (出現回数: {min_count})")
  return

print('***train set***')
count_labels(train_labels)
print('***test set***')
count_labels(test_labels)

***train set***
最大値: 648 (出現回数: 422)
最小値: 483 (出現回数: 17)
***test set***
最大値: 648 (出現回数: 106)
最小値: 506 (出現回数: 4)


In [ ]:
def max_text_length(text_list):
    if not text_list:  # 空リストの場合
        return 0

    max_length = max(len(text) for text in text_list)
    return max_length

print('***ALL***')
max_length = max_text_length(docs_filtered)
print(f"テキストの最大長は {max_length} 文字です。")

print('***train set***')
max_length = max_text_length(train_docs)
print(f"テキストの最大長は {max_length} 文字です。")

print('***test set***')
max_length = max_text_length(test_docs)
print(f"テキストの最大長は {max_length} 文字です。")


***ALL***
テキストの最大長は 127 文字です。
***train set***
テキストの最大長は 127 文字です。
***test set***
テキストの最大長は 122 文字です。


In [ ]:
#Code数のカウント
unique_count = len(set(labels_filtered))
print("ユニークな要素の数:", unique_count)

ユニークな要素の数: 1529


# モデル読み込み

In [ ]:
# GPU 利用可の場合、GPUを利用
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
model_name = "cl-tohoku/bert-base-japanese-v3"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=unique_count)
model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

for name, param in model.named_parameters():
    if 'bert.pooler.' in name:
        param.requires_grad = True
    elif 'classifier' in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cl-tohoku/bert-base-japanese-v3 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32768, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
encodings = tokenizer(docs_filtered, return_tensors='pt',padding=True).to(device)
max_token_count = encodings.input_ids.size(1)
print("       最大トークン数:", max_token_count)

train_encodings = tokenizer(train_docs, return_tensors='pt', padding='max_length', max_length=max_token_count).to(device)
test_encodings = tokenizer(test_docs, return_tensors='pt', padding='max_length', max_length=max_token_count).to(device)

# トークン化された後の最大トークン数を取得
max_token_count_tr = train_encodings.input_ids.size(1)  # 入力テンソルのトークン数を取得
print("trainの最大トークン数:", max_token_count_tr)
max_token_count_ts = test_encodings.input_ids.size(1)  # 入力テンソルのトークン数を取得
print(" testの最大トークン数:", max_token_count_ts)

# Note: 最大トークンが大きくなりすぎるようであればtruncationとMax_lengthを手動設定する

train_encodingsトークン化後の最大トークン数: 65
 test_encodingsトークン化後の最大トークン数: 58
                     すべての最大トークン数: 65


In [ ]:
class JpSentiDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = JpSentiDataset(train_encodings, train_labels)
test_dataset = JpSentiDataset(test_encodings, test_labels)

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# 学習処理

In [ ]:
import math

num_epoch = 16
batch_size = 64
lr_rate = 1e-3
num_warmup_steps  = math.ceil(len(train_docs) / batch_size) # 1 epochをウォームアップに使用


In [ ]:
training_args = TrainingArguments(
    output_dir='output/',      # output folder
    num_train_epochs=num_epoch,              # total number of training epochs
    per_device_train_batch_size=batch_size,  # batch size per device during training. Batch=512 as basis. Not done for mini batch
    per_device_eval_batch_size=4096,   # batch size for evaluation, Batch=512 as basis.
    warmup_steps=num_warmup_steps,                 # number of warmup steps for learning rate scheduler
    learning_rate= lr_rate,              # Best rate among 0.001 etc..
    #weight_decay=0.01,                # strength of weight decay
    save_total_limit=16,               # limit the checkpoints. Delete old checkpoints
    save_strategy='epoch',
    dataloader_pin_memory=False,      # Whether you want to pin memory in data loaders or not. Will default to True
    #evaluation_strategy="steps",     # Evaluate at the end of each step
    evaluation_strategy="epoch",      # Evaluate at the end of each epoch
    logging_steps=1,
    logging_dir='./logs',
    #,    overwrite_output_dir=True,  # Update files if something is available
    lr_scheduler_type="cosine",
)

trainer = Trainer(
    model=model,
    args=training_args,               # training arguments, defined above
    train_dataset=train_dataset,      # training dataset
    eval_dataset=test_dataset,        # evaluation dataset
    compute_metrics=compute_metrics   # The function that will be used to compute metrics at evaluation
)
trainer.train()

<ipython-input-22-e9b873686315>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,2.407900,2.012866,0.549726,0.521323,0.618995,0.549726
2,1.576900,0.974252,0.756300,0.748413,0.807606,0.756300
3,1.097300,0.742276,0.809564,0.803577,0.841349,0.809564


ストリーミング出力は最後の 5000 行に切り捨てられました。
<ipython-input-22-e9b873686315>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-e9b873686315>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-e9b873686315>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-22-e9b873686315>:7: UserWa

In [ ]:
model.save_pretrained('output/')
tokenizer.save_pretrained ('output/')

In [ ]:
# ダウンロードしたいフォルダを zip 圧縮する
!zip -r /content/output.zip /content/output

# 圧縮した zip ファイルをダウンロードする
#from google.colab import files
#files.download("/content/output.zip")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')